# Streaming estructurado con Azure Databricks en Power BI y Cosmos DB para el despliegue y análisis de tweets 

####Fuente:

Structured Streaming with Azure Databricks into Power BI & Cosmos DB

https://github.com/giulianorapoz/DatabricksStreamingPowerBI

In [1]:
%load_ext watermark
%watermark -a "Christian Castro" -u -d -p numpy,pandas,matplotlib
%watermark -a "Propiedad de DataIntelligence"

Christian Castro 
last updated: 2020-04-24 

numpy 1.18.1
pandas 1.0.1
matplotlib 3.1.3
Propiedad de DataIntelligence


En este notebook nos basaremos en el concepto de Streaming estructurado con Databricks y en el cómo se puede conectar directamente para utilizarlo junto con Power BI y Cosmos DB, lo que permite la visualización y análisis avanzados para una mayor disección de los datos consumidos por streaming estructurados. Construiremos una ruta de consumo de datos directamente con Azure Databricks que nos permitirá transmitir datos a un clúster de Apache Spark en tiempo casi real. Mostraremos algunas de las capacidades de análisis a las que se puede llamar directamente desde Databricks utilizando la API de Text Analytics, luego conectaremos Databricks directamente a Power BI para análisis e informes de disección de datos adicionales. Como paso final, leeremos y escribiremos desde Databricks directamente en CosmosDB como almacenamiento persistente y uso posterior.

El primer paso es configurar todos nuestros recursos individuales. Necesitaremos lo siguiente:

*    Un espacio de trabajo de Databricks y Apache spark cluster t. (Para ejecutar nuestros nuestros cuadernos).
*    Un centro de eventos, f. (Para que Databricks envíe los datos).
*    Una cuenta de servicios cognitivos t. (Para acceder a la API de Text Analytics).
*    Una aplicación de Twitter para los datos. (Para proporcionar la transmisión actual de datos).
*    Una base de datos CosmosDB como. (Para almacenar datos de forma persistente. Los datos)
*    Power BI Desktop para visualización de datos t. (Para visualizar y analizar los datos). 
     
Event Hub En primer lugar, cree un Event Hub buscando Event Hubs en Azure. En Crear espacio de nombres, elija un espacio de nombres y seleccione una suscripción de Azure, un grupo de recursos y una ubicación para crear el recurso.